In [3]:
import tensorflow as tf
import tensorflow_text as text # https://github.com/tensorflow/text/issues/192
import numpy as np
import pandas as pd

import os

from IPython import display

In [4]:
path = "rt-polaritydata/rt-polaritydata/"

pos_path = os.path.join(path, 'rt-polarity.pos')
neg_path = os.path.join(path, 'rt-polarity.neg')
print(pos_path, neg_path)

rt-polaritydata/rt-polaritydata/rt-polarity.pos rt-polaritydata/rt-polaritydata/rt-polarity.neg


In [15]:
def load_review(path, is_pos=True):
    with open(path, 'r', encoding='latin-1') as f:
        review = pd.DataFrame({'review':f.read().splitlines()})
    review['sentiment'] = 1 if is_pos else 0
    return review

In [16]:
pos_review = load_review(pos_path, is_pos=True)
neg_review = load_review(neg_path, is_pos=False)

all_reviews = pd.concat([pos_review, neg_review])
all_reviews.head()

,review,sentiment
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [17]:
all_reviews.review_split = all_reviews.review.map(lambda line: line.split())

/home/robert/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [18]:
from gensim.models import Word2Vec, KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

ModuleNotFoundError: No module named 'gensim'

In [ ]:
model

In [ ]:
queen = model['king'] - model['man'] + model['woman']
model.similar_by_word(queen)

### LSTM model

In [19]:
import operator
from functools import reduce

def create_vocabulary(df):
    splitted_reviews = df.review.map(lambda line: line.split())
    vocab = set(reduce(operator.add, splitted_reviews))
    return list(vocab)

vocab = create_vocabulary(all_reviews)

VOCAB_SIZE = len(vocab)
print('Total vocabluary size: {} words'.format(VOCAB_SIZE))

Total vocabluary size: 21401 words


In [20]:
word_to_number = {vocab[i]: i for i in range(VOCAB_SIZE)}
number_to_word = {i:vocab[i] for i in range(VOCAB_SIZE)}

In [21]:
df = all_reviews.copy()
df["review_splitted"] = df.review.map(lambda line: line.split())
df.head()

,review,sentiment,review_splitted
0,the rock is destined to be the 21st century's ...,1,"[the, rock, is, destined, to, be, the, 21st, c..."
1,"the gorgeously elaborate continuation of "" the...",1,"[the, gorgeously, elaborate, continuation, of,..."
2,effective but too-tepid biopic,1,"[effective, but, too-tepid, biopic]"
3,if you sometimes like to go to the movies to h...,1,"[if, you, sometimes, like, to, go, to, the, mo..."
4,"emerges as something rare , an issue movie tha...",1,"[emerges, as, something, rare, ,, an, issue, m..."


In [22]:
df["review_enc"] = df.review_splitted.map(lambda line: \
                                          [word_to_number[word] for word in line])

In [23]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset_tf, test_dataset = dataset['train'], dataset['test']

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [25]:
from sklearn.model_selection import train_test_split

X = df.review_enc.values.tolist()
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), y_train.shape, len(X_test), y_test.shape

(8556, (8556,), 2139, (2139,))

In [67]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE, oov_token='<UNK>')
tokenizer.fit_on_texts(df.review_splitted)


#convert text data to numerical indexes
rain_seqs=tokenizer.texts_to_sequences(df.review_splitted)
# test_seqs=tokenizer.texts_to_sequences(test_df['text'])


In [69]:
SEQ_LEN = max([len(x) for x in X_test]) + 5

train_seqs=tf.keras.preprocessing.sequence.pad_sequences(rain_seqs, maxlen=SEQ_LEN, padding="post")
# test_seqs=tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=SEQ_LEN, padding="post")

In [74]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [75]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [76]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 64)          1369664   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 1,444,033
Trainable params: 1,444,033
Non-trainable params: 0
_________________________________________________________________


In [78]:
history = model.fit(train_seqs, 
                    df.sentiment.values,
                    batch_size=BATCH_SIZE,
                    epochs=10,
                    validation_split=0.1)

Train on 10695 samples
Epoch 1/10
   32/10695 [..............................] - ETA: 3:13

AttributeError: module 'tensorflow.python.ops.gradients_util' has no attribute 'IsTrainable'